In [ ]:
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.callbacks import Callback
from keras import metrics
from sklearn.impute import KNNImputer

In [ ]:
with open('/content/dailymonthly.csv', encoding='utf-8-sig') as file_obj:
    # Create reader object by passing the file object to DictReader method
    reader_obj = csv.DictReader(file_obj)

    # Open a file to save the results
    with open("/content/GRU_NOAA_SPEI.csv", "w+") as f:
        # Define a csv writer object
        writer = csv.writer(f)
        writer.writerow(['Loc', 'mse', 'rmse', 'mae'])

        # Iterate over each row in the csv file using reader object
        for row in reader_obj:
            data = row['daily']
            print(row)

            imd = pd.read_csv('/content/NOAA/' + data + '.csv').drop(['Unnamed: 0', 'X', 'Year', 'Month'], axis=1)
            imd = imd.replace([np.inf, -np.inf], np.nan)

            X = imd.iloc[5:, 0:4]
            Y = imd.iloc[5:, 3:4]

            imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
            imputer.fit(Y)
            Y = imputer.transform(Y)
            X['Spei6'] = Y
            Y = X.iloc[:, 3]
            Ya = Y.iloc[10:]

            X_train = np.zeros((400, 20))
            X_test = np.zeros((100, 20))
            X_test_slices = []
            for i in range(400):
                X_train[i, :] = Y[i:i + 20]
            for i in range(100):
                end_index = min(len(Y) - 1, i + 400 + 20)
                slice_data = Y[i + 400:end_index]
                if len(slice_data) == 20:
                    X_test_slices.append(slice_data)
            X_test = np.array(X_test_slices)

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            Y_train = np.array(Ya.iloc[0:400])
            Y_train = Y_train.reshape(Y_train.shape[0], 1, 1)

            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
            Y_test = np.array(Ya.iloc[400:])
            Y_test = Y_test.reshape(Y_test.shape[0], 1, 1)

            model = Sequential()
            model.add(GRU(25, return_sequences=True,dropout=0.1))
            model.add(GRU(35, return_sequences=True,dropout=0.1))
            model.add(GRU(45, return_sequences=True,dropout=0.1))
            model.add(Dense(8))
            model.add(Dense(1))
            #model detals can be changed as per requirement.

            model.compile(
                loss='mae',
                optimizer=tf.keras.optimizers.legacy.Adam(
                    learning_rate=0.003,
                    decay=0.0
                    ),
                metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mse', 'mae']
                )

            history = model.fit(
                X_train,
                Y_train,
                batch_size=32,
                epochs=10,
                verbose=1,
                callbacks=[]
                )
            Y_pred = model.predict(X_test)

            mse = history.history['mse'][-1]  # Mean Squared Error
            rmse = np.sqrt(mse)  # Root Mean Squared Error
            rmse = round(rmse , 5)
            mae = history.history['mae'][-1]  # Mean Absolute Error

            # Write the results to the file
            writer.writerow([data, mse, rmse, mae])